In [ ]:
# -*- coding: utf-8 -*-

'''
Brazilian Financial Planning and Interest Comparison System (TaxCalcBR)
This script allows the user to compare different payment and investment scenarios,
such as cash vs. installment payments, impact of late payments, inflation, and alternative investments.
'''

def get_option(message, options):
    """
    Generic function to validate and get a user's option input.
    :param message: Message displayed to the user.
    :param options: List of valid options.
    :return: User's chosen option.
    """
    while True:
        try:
            option = int(input(message))
            if option in options:
                return option
            else:
                print("Invalid option! Please try again.")
        except ValueError:
            print("Invalid input! Please enter a number.")

def get_value(message):
    """
    Function to validate and get a decimal value from the user.
    :param message: Message displayed to the user.
    :return: Decimal value entered by the user.
    """
    while True:
        try:
            value = float(input(message).replace(",", "."))
            if value >= 0: # Values can be zero in some cases (e.g., 0% discount)
                return value
            else:
                print("The value must be positive or zero! Please try again.")
        except ValueError:
            print("Invalid input! Please enter a numeric value.")

def get_yes_no(message):
    """
    Function to validate and get a 'Y' or 'N' answer from the user.
    :param message: Message displayed to the user.
    :return: 'Y' or 'N' answer.
    """
    while True:
        answer = input(message).strip().upper()
        if answer in ['Y', 'N']:
            return answer
        else:
            print('Invalid answer! Please try again.')

def calculate_monthly_rate():
    """
    Calculates the monthly yield rate based on user's choice (monthly or annual).
    :return: Monthly yield rate in decimal format.
    """
    option = get_option('Choose the yield rate type: \n[1] Monthly\n[2] Annual\nOption: ', [1, 2])
    if option == 1:
        return get_value('Enter the monthly yield rate (%): ') / 100
    else:
        annual_rate = get_value('Enter the annual yield rate (%): ') / 100
        return (1 + annual_rate)**(1 / 12) - 1

def adjust_rate_for_income_tax(rate, num_installments):
    """
    Adjusts the monthly yield rate based on income tax, if the user chooses to consider it.
    :param rate: Original monthly yield rate.
    :param num_installments: Number of installments.
    :return: Adjusted rate considering or not income tax.
    """
    consider_ir = get_yes_no('Do you want to consider income tax? [Y/N]: ')

    if consider_ir == 'Y':
        if num_installments <= 6:
            return rate * 0.775  # 22.5% IR
        elif num_installments <= 12:
            return rate * 0.80  # 20% IR
        elif num_installments <= 24:
            return rate * 0.825  # 17.5% IR
        else:
            return rate * 0.85  # 15% IR
    else:
        print("The yield rate will be used without income tax adjustments.")
        return rate

def calculate_installment_value(initial_value, num_installments):
    """
    Calculates the total value and value of each installment with possible interest.
    :param initial_value: Initial purchase value.
    :param num_installments: Number of installments.
    :return: Adjusted total value and value of each installment.
    """
    has_interest = get_yes_no('Is there interest for paying in installments? [Y/N]: ')

    if has_interest == 'Y':
        interest_type = get_option('Choose the interest type: \n[1] Simple\n[2] Compound\nOption: ', [1, 2])
        if interest_type == 1:
            interest_rate = get_value('What is the interest percentage for paying in installments? ')
            total_value = initial_value * (1 + (interest_rate / 100))
        else:
            interest_period = get_option('Choose the interest period: \n[1] Daily\n[2] Monthly\n[3] Annual\nOption: ', [1, 2, 3])
            if interest_period == 1:
                daily_interest = get_value('What is the daily interest percentage (a.d.)? ') / 100
                monthly_interest = (1 + daily_interest)**30 - 1
            elif interest_period == 2:
                monthly_interest = get_value('What is the monthly interest percentage (a.m.)? ') / 100
            else:
                annual_interest = get_value('What is the annual interest percentage (a.a.)? ') / 100
                monthly_interest = (1 + annual_interest)**(1 / 12) - 1

            total_value = initial_value * (1 + monthly_interest)**num_installments
    else:
        total_value = initial_value

    return total_value, total_value / num_installments

def simulate_yield(num_installments, installment_value, monthly_rate):
    """
    Simulates accumulated yields with monthly installments.
    :param num_installments: Number of installments.
    :param installment_value: Value of each installment.
    :param monthly_rate: Adjusted monthly yield rate.
    :return: Accumulated yields.
    """
    gain = 0
    remaining_debt = installment_value * num_installments
    print(f'\n{"Installment":^10}{"Debt (R$)":>20}{"Yield (R$)":>20}')
    print('-' * 50)
    for i in range(1, num_installments + 1):
        yield_amount = remaining_debt * monthly_rate
        gain += yield_amount
        remaining_debt -= installment_value
        print(f'{i:^10}{remaining_debt:>20.2f}{yield_amount:>20.2f}')
    return gain

def compare_installment_with_down_payment():
    """
    [3] Installment with down payment: Down payment + installments vs cash
    """
    print('\n[3] Installment with Down Payment')
    down_payment = get_value('Enter the down payment amount: R$')
    installments = int(get_value('Enter the number of installments: '))
    installment_value = get_value('Enter the value of each installment: R$')
    total_value = down_payment + (installments * installment_value)
    cash_value = get_value('Enter the total cash purchase value: R$')

    print(f'\nTotal installment value with down payment: R${total_value:.2f}')
    print(f'Cash value: R${cash_value:.2f}')

    if total_value < cash_value:
        print('It's better to pay with a down payment and installments.')
    else:
        print('It's better to pay in cash.')

def compare_two_installment_options():
    """
    [4] Compare two installment options
    """
    print('\n[4] Compare Two Installment Options')
    print('First option:')
    installments1 = int(get_value('Number of installments: '))
    value1 = get_value('Value of each installment: R$')
    total1 = installments1 * value1

    print('\nSecond option:')
    installments2 = int(get_value('Number of installments: '))
    value2 = get_value('Value of each installment: R$')
    total2 = installments2 * value2

    print(f'\nTotal of the first option: R${total1:.2f}')
    print(f'Total of the second option: R${total2:.2f}')

    if total1 < total2:
        print('The first option is better.')
    else:
        print('The second option is better.')

def calculate_late_payment_impact():
    """
    [5] Impact of late payments on installments
    """
    print('\n[5] Impact of Late Payments on Installments')
    installment_value = get_value('Enter the original installment value: R$')
    days_late = int(get_value('Number of days late: '))
    penalty = get_value('Late payment penalty (%): ') / 100
    daily_interest = get_value('Daily interest for late payment (%): ') / 100

    corrected_value = installment_value * (1 + penalty + daily_interest * days_late)

    print(f'Value with late payment: R${corrected_value:.2f}')

def simulate_with_inflation():
    """
    [6] Simulation with inflation
    """
    print('\n[6] Simulation with Inflation')
    installments = int(get_value('Enter the number of installments: '))
    installment_value = get_value('Value of each installment: R$')
    inflation = get_value('Estimated monthly inflation (%): ') / 100

    real_value = 0
    for i in range(1, installments + 1):
        # Present value of each installment, discounted by accumulated inflation
        updated_value = installment_value / ((1 + inflation) ** i)
        real_value += updated_value

    print(f'Total value considering inflation (Net Present Value): R${real_value:.2f}')

def compare_with_alternative_investment():
    """
    [7] Compare with alternative investment
    """
    print('\n[7] Compare with Alternative Investment')
    cash_value = get_value('Enter the cash purchase value: R$')
    installments = int(get_value('Enter the number of installments: '))
    installment_value = get_value('Enter the value of each installment: R$')
    invest_rate = get_value('Enter the monthly investment yield rate (%): ') / 100

    # Simulates the scenario where the money NOT paid in cash is invested,
    # and installments are paid from this balance, which continues to yield.
    # The initial balance here is the value you "would have" if you paid in cash.
    # If you choose installments, this value is what you can invest.
    investment_balance = cash_value

    print(f'\n{"Month":<5}{"Initial Balance (R$)":>20}{"Yield (R$)":>18}{"Installment Paid (R$)":>20}{"Final Balance (R$)":>20}')
    print('-' * 85)

    for i in range(1, installments + 1):
        yield_amount = investment_balance * invest_rate
        investment_balance += yield_amount # The investment yields
        investment_balance -= installment_value # Pay the installment

        print(f'{i:<5}{cash_value if i==1 else investment_balance + installment_value - yield_amount:>20.2f}'
              f'{yield_amount:>18.2f}{installment_value:>20.2f}{investment_balance:>20.2f}')

    print(f'\nFinal value if you invest the cash amount and pay in installments: R${investment_balance:.2f}')
    if investment_balance > 0:
        print('Investing and paying in installments is advantageous (you end up with extra money).')
    else:
        print('Paying in cash is safer (avoids debt or losses).')


def main():
    """
    Main function to run the financial comparisons of TaxCalcBR.
    """
    print('Welcome to TaxCalcBR: Financial Planning and Interest Comparison System')

    op = get_option('''Choose the desired comparison:
[1] Yield vs Installments: Includes interest rates and discounts
[2] Cash vs Installments: Comparison between offer values
[3] Installment with down payment: Down payment + installments vs cash
[4] Compare two installment options
[5] Impact of late payments on installments
[6] Simulation with inflation
[7] Compare with alternative investment
Option: ''', [1, 2, 3, 4, 5, 6, 7])


    if op == 1:
        print('\n[1] Yield vs Installments')
        monthly_rate = calculate_monthly_rate()
        initial_value = get_value('Enter the total purchase value: R$')
        num_installments = int(get_value('Enter the number of installments: '))
        monthly_rate = adjust_rate_for_income_tax(monthly_rate, num_installments)
        print(f'The adjusted yield rate is: {monthly_rate:.4f}')

        cash_discount_available = get_yes_no('Is there a discount for cash payment? [Y/N]: ')

        if cash_discount_available == 'Y':
            discount = get_value('What is the discount percentage for cash payment? ')
            cash_value = initial_value * (1 - discount / 100)
        else:
            cash_value = initial_value

        total_installment_value, installment_per_month = calculate_installment_value(initial_value, num_installments)
        yields = simulate_yield(num_installments, installment_per_month, monthly_rate)

        effective_installment_value = total_installment_value - yields

        print(f'\nValue paying in cash: R${cash_value:.2f}')
        print(f'Value paying in installments: R${total_installment_value:.2f}')
        print(f'Accumulated yields with installment payment: R${yields:.2f}')
        print(f'Value paid in installments (minus yields): R${effective_installment_value:.2f}')

        if cash_value < effective_installment_value:
            print('\nIt's better to pay in cash.')
        else:
            print('\nIt's better to pay in installments.')

    elif op == 2:
        print('\n[2] Cash vs Installments')
        cash_value = get_value('Enter the cash purchase value: R$')
        num_installments = int(get_value('Enter the number of installments: '))
        installment_value = get_value('Enter the value of each installment: R$')
        total_installment_value = num_installments * installment_value
        monthly_rate = calculate_monthly_rate()
        monthly_rate = adjust_rate_for_income_tax(monthly_rate, num_installments)

        yields = simulate_yield(num_installments, installment_value, monthly_rate)
        effective_installment_value = total_installment_value - yields

        print(f'\nValue paying in cash: R${cash_value:.2f}')
        print(f'Value paying in installments: R${total_installment_value:.2f}')
        print(f'Accumulated yields with installment payment: R${yields:.2f}')
        print(f'Value paid in installments (minus yields): R${effective_installment_value:.2f}')

        if cash_value < effective_installment_value:
            print('\nIt's better to pay in cash.')
        else:
            print('\nIt's better to pay in installments.')

    elif op == 3:
        compare_installment_with_down_payment()
    elif op == 4:
        compare_two_installment_options()
    elif op == 5:
        calculate_late_payment_impact()
    elif op == 6:
        simulate_with_inflation()
    elif op == 7:
        compare_with_alternative_investment()


if __name__ == "__main__":
    main()
     

# -*- coding: utf-8 -*-

# Original program kept unchanged, new features added at the end as new options in the main menu

def get_float_input(prompt):
    """Helper function to get float input with error handling and comma replacement."""
    while True:
        try:
            value = float(input(prompt).replace(',', '.'))
            return value
        except ValueError:
            print('Invalid input. Please enter a numeric value.')

def get_int_input(prompt):
    """Helper function to get integer input with error handling."""
    while True:
        try:
            value = int(input(prompt))
            return value
        except ValueError:
            print('Invalid input. Please enter an integer.')

op = get_int_input('''Choose the desired comparison:
[1] Yield vs Installments: Includes interest rates and discounts
[2] Cash vs Installments: Comparison between offer values
[3] Installment with down payment: Down payment + installments vs cash
[4] Compare two installment options
[5] Impact of late payments on installments
[6] Simulation with inflation
[7] Compare with alternative investment
Option: ''')

while op not in [1, 2, 3, 4, 5, 6, 7]:
    print('Invalid option! Try again.')
    op = get_int_input('Option: ')

# Options 1 and 2 preserved intact above in this code, continue from op == 3
# *** IMPORTANT: Code for options 1 and 2 should be added here for a complete program ***

if op == 3:
    print('\n[3] Installment with down payment')
    down_payment = get_float_input('Enter the down payment amount: $')
    installments = get_int_input('Enter the number of installments: ')
    installment_value = get_float_input('Enter the value of each installment: $')
    total_value = down_payment + (installments * installment_value)
    cash_value = get_float_input('Enter the total cash purchase value: $')

    print(f'\nTotal installment value: ${total_value:.2f}')
    print(f'Cash value: ${cash_value:.2f}')

    if total_value < cash_value: # Changed logic to directly reflect "better"
        print('It is better to pay with down payment and installments.')
    else:
        print('It is better to pay in cash.')

elif op == 4:
    print('\n[4] Compare two installment options')
    print('First option:')
    installments1 = get_int_input('Number of installments: ')
    value1 = get_float_input('Value of each installment: $')
    total1 = installments1 * value1

    print('\nSecond option:')
    installments2 = get_int_input('Number of installments: ')
    value2 = get_float_input('Value of each installment: $')
    total2 = installments2 * value2

    print(f'\nTotal of the first option: ${total1:.2f}')
    print(f'Total of the second option: ${total2:.2f}')

    if total1 < total2:
        print('The first option is better.')
    else:
        print('The second option is better.')

elif op == 5:
    print('\n[5] Impact of late payments on installments')
    installment_value = get_float_input('Enter the original installment value: $')
    days_late = get_int_input('Number of days late: ')
    penalty = get_float_input('Late payment penalty (%): ') / 100
    daily_interest = get_float_input('Daily interest for late payment (%): ') / 100

    corrected_value = installment_value * (1 + penalty + daily_interest * days_late)

    print(f'Value with late payment: ${corrected_value:.2f}')

elif op == 6:
    print('\n[6] Simulation with inflation')
    installments = get_int_input('Enter the number of installments: ')
    installment_value = get_float_input('Value of each installment: $')
    inflation = get_float_input('Estimated monthly inflation (%): ') / 100

    real_value = 0
    for i in range(1, installments + 1):
        updated_value = installment_value / ((1 + inflation) ** i)
        real_value += updated_value

    print(f'Total value considering inflation: ${real_value:.2f}')

elif op == 7:
    print('\n[7] Compare with alternative investment')
    cash_value = get_float_input('Enter the cash purchase value: $')
    installments = get_int_input('Enter the number of installments: ')
    installment_value = get_float_input('Enter the value of each installment: $')
    invest_rate = get_float_input('Enter the monthly investment yield rate (%): ') / 100

    balance = cash_value # Assuming cash_value is the initial investment capital
    for i in range(installments):
        yield_value = balance * invest_rate
        balance += yield_value - installment_value

    print(f'\nFinal value if you invest and pay in installments: ${balance:.2f}')
    if balance > 0:
        print('Investing and paying in installments is advantageous.')
    else:
        print('Paying in cash is safer.')
     